In [1]:
import pandas as pd

import matplotlib.pyplot as plt

import datetime
from price_loaders.tradingview import load_asset_price
import pandas_ta as ta
from datetime import datetime, timedelta
import time
import requests


In [2]:
def send_telegram(text: str):
    token = "7724948126:AAE7vzbSwnUNWWszFsvUDSPKorR4jphXYOs"
    url = "https://api.telegram.org/bot"
    channel_id = "-1002312259096"
    url += token
    method = url + "/sendMessage"

    r = requests.post(method, data={
        "chat_id": channel_id,
        "text": text
    })

    if r.status_code != 200:
        raise Exception("post_text error")

In [3]:
def upper_strat(df, threshold=0.003):
    """
    Стратегия повышения цены (upper_strat).
    
    Сравнивает текущую цену закрытия с предыдущей.
    Если прирост цены больше или равен заданному порогу (threshold), возвращает True и значение прироста.
    В противном случае возвращает False и значение прироста.
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param threshold: Порог прироста цены.
    :return: Кортеж (булево значение, прирост цены).
    """
    initial_price = df['close'].iloc[-1] # Последняя наша цена
    prev_price = df['close'].iloc[-2] # Предыдущая цена от последней цены
    if (initial_price - prev_price) / prev_price >= threshold:
        return True, (initial_price - prev_price) / prev_price  # Возвращаем True и изменение цены
    return False, (initial_price - prev_price) / prev_price # Возвращаем False и изменение цены

def down_strat(df, threshold=0.003):
    """
    Стратегия понижения цены (down_strat).
    
    Сравнивает текущую цену закрытия с предыдущей.
    Если снижение цены больше или равно заданному порогу (threshold), возвращает True и значение снижения.
    В противном случае возвращает False и значение снижения.
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param threshold: Порог снижения цены.
    :return: Кортеж (булево значение, снижение цены).
    """
    initial_price = df['close'].iloc[-1] # Последняя наша цена
    prev_price = df['close'].iloc[-2] # Предыдущая цена от последней цены
    if (prev_price - initial_price) / prev_price >= threshold:
        return True, (prev_price - initial_price) / prev_price  # Возвращаем True и индекс времени, когда произошло падение
    return False, (prev_price - initial_price) / prev_price

def ema_strat(df, short_period = 10, long_period = 40):
    """
    Стратегия на основе экспоненциальных скользящих средних (EMA).
    
    Сравнивает короткую и длинную EMA.
    Если короткая EMA пересекает длинную EMA снизу вверх, возвращает сигнал "Покупка".
    Если короткая EMA пересекает длинную EMA сверху вниз, возвращает сигнал "Продажа".
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param short_period: Период для короткой EMA.
    :param long_period: Период для длинной EMA.
    :return: Сигнал "Покупка", "Продажа" или None.
    """
    df_copy = df.copy() # Копируем наш основной датафрейм по выбранному тикеру
    df_copy[f'EMA_{short_period}'] = ta.ema(df_copy['close'], length=short_period) # Создаем через Pandas_ta короткую EMA
    df_copy[f'EMA_{long_period}'] = ta.ema(df_copy['close'], length=long_period) # Создаем через Pandas_ta длинную EMA
    
    if df_copy[f'EMA_{short_period}'].iloc[-1] > df_copy[f'EMA_{long_period}'].iloc[-1] and \
    df_copy[f'EMA_{short_period}'].iloc[-2] <= df_copy[f'EMA_{long_period}'].iloc[-2]:
        return 'Покупка'
    
    elif df_copy[f'EMA_{short_period}'].iloc[-1] < df_copy[f'EMA_{long_period}'].iloc[-1] and \
    df_copy[f'EMA_{short_period}'].iloc[-2] >= df_copy[f'EMA_{long_period}'].iloc[-2]:
        return 'Продажа'
    
    else:
        return None
    
def range_strat(df, long_period = 120):
    """
    Стратегия на основе простой скользящей средней (SMA).
    
    Сравнивает текущую цену закрытия с длинной SMA.
    Если цена пересекает SMA снизу вверх, возвращает сигнал "Покупка".
    Если цена пересекает SMA сверху вниз, возвращает сигнал "Продажа".
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param long_period: Период для длинной SMA.
    :return: Сигнал "Покупка", "Продажа" или None.
    """
    df_copy = df.copy()
    df_copy[f'SMA_{long_period}'] = ta.sma(df_copy['close'], length=long_period)
    
    if df_copy['close'].iloc[-1] > df_copy[f'SMA_{long_period}'].iloc[-1] and \
    df_copy['close'].iloc[-2] <= df_copy[f'SMA_{long_period}'].iloc[-2]:
        return 'Покупка'
    
    elif df_copy['close'].iloc[-1] < df_copy[f'SMA_{long_period}'].iloc[-1] and \
    df_copy['close'].iloc[-2] >= df_copy[f'SMA_{long_period}'].iloc[-2]:
        return 'Продажа'
    else:
        return None
    
def rsi_strat(df, period=14, overbought_threshold=70, oversold_threshold=30):
    """
    Стратегия, основанная на индексе относительной силы (RSI).
    
    Если RSI превышает уровень перекупленности (overbought_threshold), возвращается сигнал "Продажа".
    Если RSI опускается ниже уровня перепроданности (oversold_threshold), возвращается сигнал "Покупка".
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param period: Период для расчета RSI.
    :param overbought_threshold: Уровень перекупленности для RSI.
    :param oversold_threshold: Уровень перепроданности для RSI.
    :return: Сигнал "Покупка", "Продажа" или None.
    """
    df_copy = df.copy()
    df_copy['RSI'] = ta.rsi(df_copy['close'], length=period)
    
    if df_copy['RSI'].iloc[-1] > overbought_threshold and df_copy['RSI'].iloc[-2] <= overbought_threshold:
        return 'Продажа'
    elif df_copy['RSI'].iloc[-1] < oversold_threshold and df_copy['RSI'].iloc[-2] >= oversold_threshold:
        return 'Покупка'
    else:
        return None
    
def bollinger_bands_strat(df, period=20, std_dev=2):
    """
    Стратегия на основе индикатора полос Боллинджера (Bollinger Bands).
    
    Сравнивает текущую цену закрытия с верхней и нижней полосами Боллинджера.
    Если цена закрытия ниже нижней полосы Боллинджера, возвращает сигнал "Покупка".
    Если цена закрытия выше верхней полосы Боллинджера, возвращает сигнал "Продажа".
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param period: Период для расчета SMA и полос Боллинджера.
    :param std_dev: Количество стандартных отклонений для расчета полос Боллинджера.
    :return: Сигнал "Покупка", "Продажа" или None.
    """
    df_copy = df.copy()
    df_copy['SMA'] = ta.sma(df_copy['close'], length=period)
    df_copy['Upper_Band'] = df_copy['SMA'] + (df_copy['close'].rolling(window=period).std() * std_dev)
    df_copy['Lower_Band'] = df_copy['SMA'] - (df_copy['close'].rolling(window=period).std() * std_dev)
    
    if df_copy['close'].iloc[-1] < df_copy['Lower_Band'].iloc[-1] and df_copy['close'].iloc[-2] >= df_copy['Lower_Band'].iloc[-2]:
        return 'Покупка'
    
    elif df_copy['close'].iloc[-1] > df_copy['Upper_Band'].iloc[-1] and df_copy['close'].iloc[-2] <= df_copy['Lower_Band'].iloc[-2]:
        return 'Продажа'
    else:
        return None
    
def macd_rsi_strat(df, macd_short=12, macd_long=26, macd_signal=9, rsi_period=14, rsi_overbought=70, rsi_oversold=30):
    """
    Стратегия на основе комбинации индикаторов MACD и RSI.
    
    Сравнивает сигналы MACD и RSI.
    - Покупка, если MACD пересекает сигнальную линию снизу вверх и RSI ниже уровня перепроданности.
    - Продажа, если MACD пересекает сигнальную линию сверху вниз и RSI выше уровня перекупленности.
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param macd_short: Короткий период для MACD.
    :param macd_long: Длинный период для MACD.
    :param macd_signal: Период сигнальной линии MACD.
    :param rsi_period: Период для расчета RSI.
    :param rsi_overbought: Уровень перекупленности для RSI.
    :param rsi_oversold: Уровень перепроданности для RSI.
    :return: Сигнал "Покупка", "Продажа" или None.
    """
    df_copy = df.copy()
    
    # Вычисляем MACD
    macd = ta.macd(df_copy['close'], fast=macd_short, slow=macd_long, signal=macd_signal)
    df_copy['MACD'] = macd[f'MACD_{macd_short}_{macd_long}_{macd_signal}']
    df_copy['MACD_Signal'] = macd[f'MACDs_{macd_short}_{macd_long}_{macd_signal}']

    
    # Вычисляем RSI
    df_copy['RSI'] = ta.rsi(df_copy['close'], length=rsi_period)
    
    # Сигнал на покупку
    if df_copy['MACD'].iloc[-1] > df_copy['MACD_Signal'].iloc[-1]  \
    and df_copy['MACD'].iloc[-2] <= df_copy['MACD_Signal'].iloc[-2] \
    and df_copy['RSI'].iloc[-1] < rsi_oversold:
        return 'Покупка'
    
    # Сигнал на продажу
    elif df_copy['MACD'].iloc[-1] < df_copy['MACD_Signal'].iloc[-1] \
    and df_copy['MACD'].iloc[-2] >= df_copy['MACD_Signal'].iloc[-2] \
    and df_copy['RSI'].iloc[-1] > rsi_overbought:
        return 'Продажа'
    
    else:
        return None

In [4]:
tickers = ['BYBIT:TONUSDT','BYBIT:BTCUSDT', 'BYBIT:ETHUSDT', 'BYBIT:TRXUSDT','BYBIT:SUIUSDT',
                'BYBIT:SOLUSDT', 'BYBIT:AXSUSDT','BYBIT:ADAUSDT'
               'BYBIT:APTUSDT', 'BYBIT:AVAXUSDT', 'BYBIT:BNBUSDT', 'BYBIT:DOGEUSDT', 'BYBIT:DOTUSDT',
                'BYBIT:DYDXUSDT', 'BYBIT:DASHUSDT', 'BYBIT:EOSUSDT', 
               'BYBIT:FLOWUSDT', 'BYBIT:GALAUSDT', 'BYBIT:LTCUSDT', 'BYBIT:LDOUSDT', 'BYBIT:KSMUSDT',
                'BYBIT:ONEUSDT', 'BYBIT:OPTUSDT', 'BYBIT:SLPUSDT', 
               'BYBIT:XRPUSDT', 'BYBIT:XMLUSDT', 'BYBIT:XMRUSDT', 'BYBIT:MKRUSDT']

In [6]:
while True:
    signals = 0 # Счетчик всех сигналов за прогон бота
    print('\n')
    
    try:
        for ticker in tickers: # Пишем цикл для обработки всех тикеров в нашем листе
            data = load_asset_price(ticker, 500, 15, None)
            data['time']=pd.to_datetime(data['time'], unit='s')

            if upper_strat(data, threshold=0.00003)[0] == True:
                print(f"Тикер: {ticker}, Повышение цены на {round(upper_strat(data, threshold=0.003)[1],4)}, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Повышение цены на {round(upper_strat(data, threshold=0.003)[1],4)}, Цена: {data['close'].iloc[-1]}")
                signals +=1
            if down_strat(data, threshold=0.00003)[0] == True:
                print(f"Тикер: {ticker}, Понижение цены на {round(down_strat(data, threshold=0.003)[1],4)}, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Понижение цены на {round(down_strat(data, threshold=0.003)[1],4)}, Цена: {data['close'].iloc[-1]}")
                signals +=1
            if ema_strat(data, short_period = 10, long_period = 40) == 'Покупка':
                print(f"Тикер: {ticker}, Покупка по двум ЕМА, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Покупка по двум ЕМА, Цена: {data['close'].iloc[-1]}")
                signals +=1
            if ema_strat(data, short_period = 10, long_period = 40) == 'Продажа':
                print(f"Тикер: {ticker}, Продажа по двум ЕМА, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Продажа по двум ЕМА, Цена: {data['close'].iloc[-1]}")
                signals +=1
    except:
        print(f"Ошибка по тикеру: {ticker}")
    
    print(f'\nПрогон завершен\nСигналов: {signals}')
    time.sleep(5)



Тикер: BYBIT:TONUSDT, Повышение цены на 0.0065, Цена: 5.388
Тикер: BYBIT:BTCUSDT, Повышение цены на 0.0017, Цена: 94490.49
Тикер: BYBIT:ETHUSDT, Повышение цены на 0.0036, Цена: 3255.44
Тикер: BYBIT:TRXUSDT, Понижение цены на 0.0002, Цена: 0.24234
Тикер: BYBIT:SUIUSDT, Повышение цены на 0.0055, Цена: 5.0177
Тикер: BYBIT:SOLUSDT, Повышение цены на 0.0004, Цена: 185.2
Тикер: BYBIT:AXSUSDT, Повышение цены на 0.0074, Цена: 6.2688
Ошибка по тикеру: BYBIT:ADAUSDTBYBIT:APTUSDT

Прогон завершен
Сигналов: 7


Тикер: BYBIT:TONUSDT, Повышение цены на 0.0004, Цена: 5.395
Тикер: BYBIT:BTCUSDT, Понижение цены на 0.0001, Цена: 94483.9
Тикер: BYBIT:BTCUSDT, Покупка по двум ЕМА, Цена: 94483.9
Тикер: BYBIT:ETHUSDT, Понижение цены на 0.0001, Цена: 3255.92
Тикер: BYBIT:TRXUSDT, Понижение цены на 0.0, Цена: 0.2424
Тикер: BYBIT:SUIUSDT, Понижение цены на 0.0002, Цена: 5.0162
Тикер: BYBIT:SOLUSDT, Повышение цены на 0.0015, Цена: 185.46
Тикер: BYBIT:AXSUSDT, Повышение цены на 0.0074, Цена: 6.2688
Ошибка по т

KeyboardInterrupt: 